In [1]:
from patchedimage_color import PatchedImageColor
from utilities import *


In [3]:
im = PatchedImageColor("images/pont.png",4,search_mode="Local")
masque = masque_circulaire((100,80),40,im.img.shape,color=True)
im.set_masque(draw=False,masque=masque,leaf_size=256)
im.set_priorities()
im.show_img(search_zone=True)

res = im.reconstruction_auto()

im.show_img()

(481, 587)
Size of the search zone : 225x245
==Tree construction==
1.123 sec


error: OpenCV(4.10.0) /private/var/folders/k1/30mswbxs7r1g6zwn8y4fyt500000gp/T/abs_49s_p64pd6/croot/opencv-suite_1722029132360/work/modules/imgproc/src/color.simd_helpers.hpp:94: error: (-2:Unspecified error) in function 'cv::impl::(anonymous namespace)::CvtHelper<cv::impl::(anonymous namespace)::Set<3, 4, -1>, cv::impl::(anonymous namespace)::Set<3, 4, -1>, cv::impl::(anonymous namespace)::Set<0, 2, 5>, cv::impl::(anonymous namespace)::NONE>::CvtHelper(cv::InputArray, cv::OutputArray, int) [VScn = cv::impl::(anonymous namespace)::Set<3, 4, -1>, VDcn = cv::impl::(anonymous namespace)::Set<3, 4, -1>, VDepth = cv::impl::(anonymous namespace)::Set<0, 2, 5>, sizePolicy = cv::impl::(anonymous namespace)::NONE]'
> Unsupported depth of input image:
>     'VDepth::contains(depth)'
> where
>     'depth' is 6 (CV_64F)
